# post quantum key exchange

In [1]:
 def GLEmb(n,m,M): 
    
    """
  This function an embedding GL(n) --> GL(N) associated to the standard Veronese Embedding of degree m, where
  N = binomial(n+m-1,m). 
  
  The map  v_{n-1,m}:P^(n-1) to P^(N-1) is the Veronese embedding.
  
 Input :
    m= degree of Veronese Embedding
    n-1 = dimension of projective space P^(n-1)
    M =   n X n Matrix acting on the coordinates of P^(n-1)
 Output:
     returns a  N x N matrix  correspoding to the action of the n X n Matrix M, associated to the
     Veronese embedding.

"""

 
    N = binomial(n+m-1,m)
    F = M.base_ring()
    P = ProjectiveSpace(F,n-1)
    L = P.veronese_embedding(m).defining_polynomials()
    A = PolynomialRing(F,n^2,'u')
    B = PolynomialRing(A,n,'v')    
    L1 = [B(L[i]) for i in range(N)]      
    v = vector(Matrix(B,n,A.gens())*Matrix(B,n,1,B.gens()))     
    L = [L[i](*v) for i in range(N)]      
    ls1 = []
    for i in range(N):
        for j in range(N):
            ls1.append(L[i].coefficient(L1[j]))
    M3 = Matrix(A,N,ls1)     
    w = vector(M)  
    M4 = Matrix(F,N)
    for i in range(N):
        for j in range(N):                
            M4[i,j] = M3[i,j](*w)        
    return M4          

In [2]:
GLEmb?

In [133]:
 """  This function translates a matrix to its corresponding projective map.
 Input: Projective Spaces Pn,Pm of dimensions n-1 and m-1 respectively and  m x n matrix  M.
 Output : Returns a Projective map f:Pn -> Pm whose defining polynomials
 are given by the action of the matrix M to the coordinates of Pn.  
 """ 
def matrix_to_projective_map(Pn,Pm,M):
    m = Pm.dimension()+1
    n = Pn.dimension()+1
    Rn = Pn.coordinate_ring()       
    if m != M.nrows() or n != M.ncols():  
        return("Incompatible values")
    M1 =  list(vector(M.change_ring(Rn)*Matrix(Rn,n,1,Rn.gens()))) # defining polynomials
    return Pn.Hom(Pm)(M1)    
       

In [134]:

"""
This function returns a matrix of a sigma embedding

Input : A sigma embedding s11:P1xP1 ->PN and the degree of veronese embedding i.e. m .
Output : The representation matrix  of the sigma embedding, which is of order \binom(m+3,m) X (m+1)^2.
"""

def matrix_of_sigma(sigma,m):
    N = binomial(m+3,3)
    F= sigma.domain().base_ring()    
    R.<x0,x1,x2,x3> = PolynomialRing(F,4,'x')
    Defsigma = sigma.defining_polynomials()
    ls = ((x0*x2+x0*x3+x1*x2+x1*x3)^m).monomials()
    ls1= []
    for i in range(N):
        for j in ls:
            ls1.append(Defsigma[i].coefficient(j)) 
    M = Matrix(F,N,(m+1)^2,ls1)    
    return M

In [135]:
""" 
This function returns a sigma embedding  P1XP1 -->PN  whose representing matrix M is given matrix.

Input : Projective spaces P1XP1,PN degree of veronese embedding m and a matrix M.
Output : Returns a sigma embedding whose representation matrix is M of order binom(m+3,m) X (m+1)^2.

"""
def sigma_of_matrix(P1XP1,m,PN,M):
    F  = P1XP1.base_ring()
    P1.<x0,x1> = ProjectiveSpace(F,1)
    P2.<x2,x3> = ProjectiveSpace(F,1)
    R.<x0,x1,x2,x3> = PolynomialRing(F,4,'x')
    phi1 = P1.veronese_embedding(m).defining_polynomials()
    phi2 = P2.veronese_embedding(m).defining_polynomials()    
    ls3= []
    for i in phi1:
        for j in phi2:
            ls3.append(R(i)*R(j))     
    L=list(vector(M.change_ring(R)*Matrix(R,(m+1)^2,1,ls3)))
    return P1XP1.Hom(PN)(L)
    

In [136]:
# J INVARIANT OF A (2,2)-CURVE
# Calulates j-invariant of a (2,2)- curve.
def j_invariant22(C):
    P1xP1=C.ambient_space()
    R11=P1XP1.coordinate_ring()
    F=P1XP1.base_ring()
    A.<u0,u1>=PolynomialRing(F,2,'u')
    B.<v0,v1,v2,v3>=PolynomialRing(A,4,'v')
    f=C.defining_polynomials()[0]    
    f=B(f)    
    f=f(u0,u1,v2,v3)
    Cf=f.coefficients()
    g=Cf[1]^2-4*Cf[0]*Cf[2]
    g=g/g([0,1])
    Cg=g.coefficients()
    S=Cg[0]-(Cg[1]*Cg[3])/4+(Cg[2]^2)/12
    T=Cg[0]*Cg[2]/6+Cg[1]*Cg[2]*Cg[3]/48-Cg[2]^3/216-Cg[0]*Cg[3]^2/16-Cg[1]^2/16
    j=S^3/(S^3-27*T^2)
    return(j)

$abc$

In [137]:
# Parameters
# parameters construction for m = 5
q=next_prime(2^14)
bfF=FiniteField(q)
m=8
N=binomial(m+3,3)
P3=ProjectiveSpace(bfF,3)
P1XP1=ProductProjectiveSpaces(bfF,[1,1])
PN=ProjectiveSpace(bfF,N-1) 
RN=PN.coordinate_ring()

In [138]:
"""
 Key generation by Alice: 
 Private  Keys are : MAs, MA(not required in key exchange, can be forgotten after the key generation)
 Public Keys are: MAp, HA
 MA: A  matrix representation of the Veronese Variety
 MAp:A  matrix representation of the public sigma embedding sigmap
 MAs:A  matrix representation of the private sigma embedding sigmas 
 HA: Hyperplane containing the image of secret sigma embedding sigmas
 
"""

def keygenA():
    vA=P3.veronese_embedding(m)      # Chooses a random N by N Matrix to define a random Veronese Variety
    
    MA=random_matrix(bfF,N)
    
    phi= matrix_to_projective_map(PN,PN,MA)          #associated projective map
    
    vA=phi*vA           # A Veronese map with representing matrix MA
    
    # We construct automorphisms of the Veronese Variety.
    
    # Choose random 4 by 4 matrices A1dash and A2dash.
    A1dash=random_matrix(bfF,4) 
    A2dash=random_matrix(bfF,4)
    
    A1=MA*GLEmb(4,m,A1dash)*MA^(-1)   # random automorphism of the Veronese variety
    A2=MA*GLEmb(4,m,A2dash)*MA^(-1)   # random automorphism of the Veronese variety
    
    
    # Now, we construct private and public sigma embeddings
    
    # Random 4 by 4 matrices.
    Ap = random_matrix(bfF,4)
    As = random_matrix(bfF,4)
    
    # Corresponding projective maps
    phip=matrix_to_projective_map(P3,P3,Ap)
    phis=matrix_to_projective_map(P3,P3,As) 
    
    segre=P1XP1.segre_embedding().defining_polynomials()
    hom = P1XP1.Hom(P3)
    segre = hom(segre)
    
    sigmap=vA*phip*segre        # The public sigma embedding
    
    sigmas=vA*phis*segre        #  The  secret sigma embedding
    
    # Matrices representing the sigma embeddings.
    
    MAp=matrix_of_sigma(sigmap,m) # matrix of public sigma embedding : Public Key = MAp
    MAs=matrix_of_sigma(sigmas,m) # matrix of secret sigma embedding: Private Key  = MAs
    
    # construction of Hyperplane containing the image of secret sigma embedding
    
    CoeffHA=MAs.kernel().basis()[0]
    HA=RN(0)
    for i in range(N):
        HA=HA+(RN(CoeffHA[i]))*RN.gens()[i]
        
    HA=PN.subscheme(HA) # Public Key: a hyperplane containing the image of the sigma embedding "sigmas"
    return sigmas, (HA, A1, A2, MAp)

In [139]:
"""
BOB COMPUTATIONS
Bob's key construction
Private Key: MB (The representation matrix for the private sigma embedding sigmaB)
Public Key:  HB (A hyperplane containing the image of the sigma embedding sigmaB)
"""


def keygenB(A1, A2, MAp):
    
    b=[Zmod(q).random_element() for i in range(4)]
    
    MBdash=A1^b[0]*A2^b[1]*A1^b[2]*A2^b[3] # construction of random automorphism of the Veronese variety .
    
    MB=MBdash*MAp
    
    sigmaB=sigma_of_matrix(P1XP1,m,PN,MB) # secret sigma embedding 
    
    CoeffHB=MB.kernel().basis()[0]
    HB=RN(0)
    for i in range(N):
        HB=HB+(RN(CoeffHB[i]))*RN.gens()[i]       
    HB = PN.subscheme(HB)
    return (sigmaB,HB)

In [140]:
"""
 Bob's computation of the Common Key
 Bob calculates the common key, a j-invariant of a (2,2)-curve from the Alice's public hyperplane = HA
 and his private sigma embedding= sigmaB

"""
def Bob_common_key(HA):
    XB=HA.defining_polynomials()[0](*sigmaB) #pullback  a 
    
    # factorization to get (2,2) curve 
    if m ==3:
        PXB=XB.factor()[1][0]
    else:
        PXB=XB.factor()[0][0] 
    CA2 = P1XP1.subscheme(PXB)
    jB=j_invariant22(CA2)
    return jB

In [141]:

"""
 ALICE's computation of the Common key
 Alice calculates the common key, a j-invariant of a (2,2)-curve from the Bob's public hyperplane = HB
 and her private sigma embedding= sigmas
"""
def Alice_common_key(HB):
    
    XA=HB.defining_polynomials()[0](*sigmas) # pullback
    
    # factorization to get (2,2) curve 
    if m ==3:
        PXA=XA.factor()[1][0]
    else:
        PXA=XA.factor()[0][0]
        
    CA1 = P1XP1.subscheme(PXA)
    
    jA=j_invariant22(CA1)
    return jA


In [142]:
%time sigmas,(HA, A1, A2, MAp) = keygenA()

CPU times: user 47.4 s, sys: 424 ms, total: 47.9 s
Wall time: 48.6 s


In [143]:
%time  (sigmaB,HB) = keygenB(A1, A2, MAp)

CPU times: user 497 ms, sys: 3.98 ms, total: 501 ms
Wall time: 602 ms


In [144]:
%time jB = Bob_common_key(HA)

CPU times: user 87.1 ms, sys: 8.05 ms, total: 95.1 ms
Wall time: 585 ms


In [145]:
%time jA = Alice_common_key(HB)

CPU times: user 71.2 ms, sys: 7.87 ms, total: 79.1 ms
Wall time: 78.4 ms


In [146]:
jA, jA==jB

(4256, True)